# Web scraping

Scraping unstructured information from the web into structured data is a common use of Python in the newsroom. In this session, we're going to use a module called [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to do most of the heavy lifting.

### Quick overview: HTML

To scrape a web page, you need to _sort of_ understand how web pages are made.

Web pages are written in HTML. HTML elements are represented (_usually_) by a pair of tags -- an opening tag and a closing tag.

A table, for example, starts with `<table>` and ends with `</table>`. The first tag tells the browser: "Hey! I got a table here! Render it as a table." The closing tag (note the forward slash) tells the browser: "Hey! I'm all done with that table, thanks." Inside the table are nested tags representing rows and cells.

(There's more to it, but that's probably good for now.)

### Inspect the source!

If I'm thinking about scraping a page, the first thing I do is look at the HTML code that underpins the page. You can do this right from your browser -- I like to use Chrome but Firefox has some good developer tools, as well. (Maybe IE does too, who knows lol)

To "view source" in Chrome, you'd hit `Ctrl+U` on a PC and `Cmd+Opt+U` on a Mac. It's also in the menu bar: View -> Developer -> View Page Source.

You'll get a page showing you all the HTML code that makes up that page.

### BeautifulSoup

BeautifulSoup turns HTML into data objects that Python can work with, allowing you to walk up and down the HTML tag "tree" and target specific elements on the page.

## Warmup

Before we start operating on a live page, let's practice on a sample file at `../practice-table.html`. Normally, the first step to scrape a web page is to fetch it -- later, we'll use `requests` for this again -- but for this example let's pretend we've already got a local copy.

The Mountain Goats [have a new album out](https://themountaingoats.bandcamp.com/album/goths) (it is good, you should buy it); the HTML we're going to operate on is just a `<table>` showing the track listing.

Let's start by importing the `BeautifulSoup` class from our module, which is called `bs4`.

Next, we're going to open the file and use the `read()` method to read its contents into memory. Then let's print the contents of the file.

Now let's feed the file contents to a BeautifulSoup object and assign the result to the variable `soup`. You might get an error unless you also pass `'html.parser'` as the second argument. Now print `type(soup)`.

Cool. We're locked and loaded. Our string of HTML is now a tree that we can climb through to find the things we want.

There are a couple of ways to isolate the table we want using the `find` or `find_all` methods -- by class, by ID, by position on the page, by style. (There are others.) Let's try:

We've targeted the correct table. Now what if we wanted to print a list of track numbers and song titles? Look at the structure of the table -- a `table` has rows represented by the tag `tr`, and within each row there are cells represented by `td`. The `find_all()` method, you'll recall, returns a _list_. And we know how to iterate over lists: with a `for` loop.

---

Now let's work on a live example.

### Scraping etiquette

**Rule No. 1: Don't hammer their servers.** If feasible, save a copy of the page locally so you only need to fetch it once as you write your script. For me, at least, scraping involves a lot of "try this and see if it works," which can involve running the same script a dozen (or hundreds) of times as you refine it. If you're requesting multiple pages, pause a for a little bit between requests so you don't overload them.

That's good for now. We can go over other rules as problems arise.

We are going to scrape [this website](http://www.nrc.gov/reactors/operating/list-power-reactor-units.html) in just Four! Easy! Steps!

1. Import libraries
2. Grab the contents of the web page
3. Parse the contents of the web page and target the data table
4. Loop over the table and write the contents to a CSV file

### 1. Import libraries

We'll need `requests` to fetch the page, `bs4` (the BeautifulSoup class, at least) to parse it, and our old friend `csv` to write out to a new file.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

### 2. Grab the contents of the web page

`requests` has a method called `get()` that -- you guessed it! -- gets a page. The `text` attribute of the result will give you the string of HTML we need to hand off to BeautifulSoup.

### 3. Parse the contents, target the data table

Lucky for us, there's only one `<table>` on the page. How do I know this? I viewed the source code and `Ctrl+F`'d for "&lt;table". So we can use the `find()` method to get it once we make soup out of that string of HTML.

### 4. Loop over the table, write the contents to a CSV

Here's where the action is:

- Write a function, `cleanString()`, that strips whitespace and garbage characters that throw off Windows
- In a `with` block, open a file to write to -- call it `reactors.csv` or something -- in `w` mode
- Define your list of headers
- Create a writer or DictWriter object
- Write headers to the file
- Loop over the rows in the table we targeted, extracting the data and writing to the file

The first cell in each row is kind of tricky: It has a `<br>` tag to break the line, so you'll need to use the [`contents`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#contents-and-children) attribute to get a tag list, then target the items in that list. You can access the `'href'` attribute of the link in the same way that you'd get a value from a dictionary -- with bracket notation.

## Next step: Scraping data from more than one page

In theory, we could have just imported that table into Excel. Let's _kick things up a notch_.

Problem: We scraped our table, but there is one key piece of information on each reactor's detail page that we want to grab, too: A link to the PDF of its operating license.

To get this link, we'll repeat the steps we just took to scrape the overview table -- but we'll tweak it a little bit, adding a function that extracts the link from each detail page.

### Extraction function!

Let's write an extraction function, `fetchLicensePDF()`, that will take one argument, `detail_html` -- the string of HTML from a reactor's detail page -- and return the extra link we wanna grab.

The PDF link is in a list inside another table cell, but we're going to target it using the link's text ("Plant Operating License"). It's a relative link, so we're going to prepend the domain before we write out to our file.

Some of the pages don't have this link, so we're going to use a [`try/except`](https://docs.python.org/3/tutorial/errors.html) statement to catch errors.

We have our functions. We're ready to write out to a new file -- let's call this one 'reactor-detail.csv'. We're going to repeat the same loop we did when we scraped the first table, except this time, we're going to to do more things:

1. Grab the detail page link and call the `scrapeDetail()` function on it, then write out the values in the returned dictionary alongside the other data
2. Pause for a couple seconds using the `time` module